#### Librerias

In [22]:
import pandas as pd
from datetime import datetime
import itertools
import numpy as np
import Proyecto as pro
import random
import math
from scipy.optimize import minimize

#### Funciones

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias entre las coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en kilómetros (aproximadamente 6371)
    R = 6371
    distancia = R * c
    
    return distancia

def combine_datetime(date, time):
    try:
        # Intentar combinar fecha y hora
        return pd.to_datetime(date + ' ' + time)
    except Exception:
        # Si hay un error (por ejemplo, hora incompleta), asignar '00:00:00'
        return pd.to_datetime(date + ' ' + '00:00:00')

def millas_a_kilometros(millas):
    return millas * 1.60934

def calcular_distancia(hora_inicio, hora_fin, velocidad_promedio=80):
    # Convertir las horas a formato datetime
    hora_inicio = pd.to_datetime(hora_inicio)
    hora_fin = pd.to_datetime(hora_fin)
    
    # Calcular la duración en horas
    duracion_horas = (hora_fin - hora_inicio).dt.total_seconds() / 3600
    
    # Calcular la distancia recorrida (distancia = velocidad promedio * tiempo)
    distancia_recorrida = velocidad_promedio * duracion_horas
    
    return distancia_recorrida

def extraer_hms(hora_inicio, hora_fin):
    diferencias = []
    
    for inicio, fin in zip(hora_inicio, hora_fin):
        diferencia = fin - inicio
        
        # Extraer horas, minutos y segundos de la diferencia
        horas = diferencia.seconds // 3600
        minutos = (diferencia.seconds % 3600) // 60
        segundos = diferencia.seconds % 60
        
        # Agregar la diferencia formateada a la lista de resultados
        diferencias.append(f"{horas:02}:{minutos:02}:{segundos:02}")
    
    return diferencias

#### Lectura de datos

In [4]:
uber = pd.read_csv('DataSets/My Uber Drives - 2016.csv')
amazon = pd.read_csv('DataSets/amazon_delivery.csv')
dataset = pd.read_csv('DataSets/dataset.csv')

#### Limpieza y transformación

In [5]:
#Para Horas
#Solo usar en la primera activacion del kernel o abra que reiniciarlo cada vez
uber['START_DATE*'] = pd.to_datetime(uber['START_DATE*'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')
uber['END_DATE*'] = pd.to_datetime(uber['END_DATE*'], format='%m/%d/%Y %H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')


temp_uber_SD = pd.to_datetime(uber['START_DATE*'])
temp_uber_ED = pd.to_datetime(uber['END_DATE*'])

temp_amazon_SD = amazon.apply(lambda row: combine_datetime(row['Order_Date'], row['Order_Time']), axis=1)
temp_amazon_ED = amazon.apply(lambda row: combine_datetime(row['Order_Date'], row['Pickup_Time']), axis=1)

temp_dataset_SD = pd.to_datetime(dataset['created_at'])
temp_dataset_ED = pd.to_datetime(dataset['actual_delivery_time'])

#En uber tengo millas
#en amazon tengo latitudes y regresa en kilometros

#Distancia
temp_uber_dist = millas_a_kilometros(uber['MILES*'])
temp_amazon_dist = haversine(amazon['Store_Latitude'], amazon['Store_Longitude'],amazon['Drop_Latitude'], amazon['Drop_Longitude'])
temp_dataset_dist = calcular_distancia(dataset['created_at'], dataset['actual_delivery_time'])

#Tiempo neto
temp_uber_time = extraer_hms(temp_uber_SD, temp_uber_ED)
temp_amazon_time = extraer_hms(temp_amazon_SD, temp_amazon_ED)
temp_dataset_time = extraer_hms(temp_dataset_SD, temp_dataset_ED)

#Categorias
temp_uber_cap = ["viaje"] * len(uber)
temp_amazon_cap = ["e-commerce"] * len(amazon)
temp_dataset_cap = ["deliveri"] * len(dataset)

#Veiculo
temp_uber_veh = ["van"] * len(uber)
temp_amazon_veh = amazon['Vehicle']
temp_dataset_veh = ["scooter "] * len(dataset)



In [6]:
len(uber)

1156

#### Union

In [8]:
fecha_inicio = [temp_amazon_SD, temp_dataset_SD, temp_uber_SD]
fecha_fin = [temp_amazon_ED, temp_dataset_ED, temp_uber_ED]
tiempo = [temp_amazon_time, temp_dataset_time, temp_uber_time]
distancia = [temp_amazon_dist, temp_dataset_dist, temp_uber_dist]
categoria = [temp_amazon_cap, temp_dataset_cap, temp_uber_cap]
vehiculo = [temp_amazon_veh, temp_dataset_veh, temp_uber_veh]



fecha_inicio_flat = list(itertools.chain(*fecha_inicio))
fecha_fin_flat = list(itertools.chain(*fecha_fin))
tiempo_flat = list(itertools.chain(*tiempo))
distancia_flat = list(itertools.chain(*distancia))
categoria_flat = list(itertools.chain(*categoria))
vehiculo_flat = list(itertools.chain(*vehiculo))

tam = np.array(fecha_inicio_flat).shape
tam = tam[0]**(1/2)
tam = round(tam)
usuarios = random.choices(range(10000000, 100000000), k=tam)
usuarios_flat = [random.choice(usuarios) for _ in range(len(fecha_inicio_flat))]

dict_temp = {
    'Fecha Inicio': fecha_inicio_flat,
    'Fecha Fin': fecha_fin_flat,
    'Usuarios': usuarios_flat,
    'Tiempo': tiempo_flat,
    'Distancia': distancia_flat,
    'Categoria': categoria_flat,
    'Vehiculo': vehiculo_flat
}

BasedeDatos = pd.DataFrame(dict_temp)
BasedeDatos = BasedeDatos.dropna(subset=['Fecha Inicio'])
BasedeDatos = BasedeDatos.dropna(subset=['Fecha Fin'])

In [9]:
BasedeDatos.head()
BasedeDatos.shape

(242315, 7)

In [10]:
#Exportar la BD
BasedeDatos.to_csv("DataSet_Final/BasedeDatos.csv", index=False)

In [11]:
Dicotomico = pro.MetodoDicotomico(BasedeDatos, 0.1, 0.5)
inter = Dicotomico.aplicarMetodo()
print(inter)

(Timedelta('0 days 00:11:41.229373932'), Timedelta('0 days 00:11:41.593596268'))


In [12]:
filtro_1 = BasedeDatos[BasedeDatos['Vehiculo'] == 'scooter ']
filtro_2 = BasedeDatos[BasedeDatos['Vehiculo'] == 'motorcycle ']
filtro_3 = BasedeDatos[BasedeDatos['Vehiculo'] == 'van']

t_1 = pd.to_timedelta(filtro_1['Tiempo']).dt.total_seconds()
d_1 = filtro_1['Distancia']
v1 = np.sum(t_1)/np.sum(d_1)

t_2 = pd.to_timedelta(filtro_2['Tiempo']).dt.total_seconds()
d_2 = filtro_2['Distancia']
v2 = np.sum(t_2)/np.sum(d_2)

t_3 = pd.to_timedelta(filtro_3['Tiempo']).dt.total_seconds()
d_3 = filtro_3['Distancia']
v3 = np.sum(t_3)/np.sum(d_3)

c = np.array([v1, v2, v3])
A = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
]) 
b = np.array([43738, 1155, 179428])

Simplex = pro.MetodoSimplex(c, A, b)
vals = Simplex.aplicarMetodo()
print(vals)

(array([0., 0., 0.]), -0.0)


### Minimizar

In [43]:
# Minimizar
# Cargar la base de datos (ajusta la ruta si es necesario)
df = pd.read_csv("DataSet_Final/BasedeDatos.csv")


In [45]:
#Separacion
# Filtrar el DataFrame original por cada categoría 
df_ecommerce = df[df['Categoria'] == 'e-commerce'] 
df_delivery = df[df['Categoria'] == 'deliveri'] 
df_viaje = df[df['Categoria'] == 'viaje']

In [55]:
# Calcular los tiempos y distancias promedio actuales para cada categoría
promedio_tiempo = {
    'e-commerce': df_ecommerce['Tiempo'].mean(),
    'delivery': df_delivery['Tiempo'].mean(),
    'transporte': df_viaje['Tiempo'].mean()
}
promedio_distancia = {
    'e-commerce': df_ecommerce['Distancia'].mean(),
    'delivery': df_delivery['Distancia'].mean(),
    'transporte': df_viaje['Distancia'].mean()
}

Minimizacion python

In [ ]:
# Asegurarnos de que los promedios no sean NaN ni cero
for key in promedio_tiempo:
    if np.isnan(promedio_tiempo[key]) or promedio_tiempo[key] == 0:
        raise ValueError(f"El promedio de tiempo para {key} es inválido: {promedio_tiempo[key]}")

# Definir la función de minimización para cada categoría
def minimizar_tiempo(categoria):
    # Asignar coeficientes aleatorios
    coef = {'c1': np.random.uniform(0.1, 1.0), 'c2': np.random.uniform(0.1, 1.0)}
    
    def objetivo(x):
        # x[0]: cantidad de servicios
        tiempo_total = x[0] * promedio_tiempo[categoria]
        return coef['c1'] * x[0] + coef['c2'] * tiempo_total
    
    # Definir las restricciones (asegurar no-negatividad)
    cons = [{'type': 'ineq', 'fun': lambda x: x[0]}]
    
    # Límites de las variables (para asegurar que sean no negativas)
    bounds = [(0, None)]
    
    # Realizar la optimización
    resultado = minimize(objetivo, x0=[1], constraints=cons, bounds=bounds, method='SLSQP', options={'disp': True})
    
    return resultado, coef

# Aplicar la minimización a cada categoría
resultado_ecommerce, coef_ecommerce = minimizar_tiempo('e-commerce')
resultado_delivery, coef_delivery = minimizar_tiempo('delivery')
resultado_transporte, coef_transporte = minimizar_tiempo('transporte')

# Imprimir los coeficientes generados y los resultados
print("Coeficientes aleatorios para e-commerce:", coef_ecommerce)
print("Resultados de la optimización para e-commerce:")
print("Estado:", resultado_ecommerce.success)
print("Valores óptimos de x:", resultado_ecommerce.x)
print("Función objetivo mínima:", resultado_ecommerce.fun)

print("\nCoeficientes aleatorios para delivery:", coef_delivery)
print("Resultados de la optimización para delivery:")
print("Estado:", resultado_delivery.success)
print("Valores óptimos de x:", resultado_delivery.x)
print("Función objetivo mínima:", resultado_delivery.fun)

print("\nCoeficientes aleatorios para transporte:", coef_transporte)
print("Resultados de la optimización para transporte:")
print("Estado:", resultado_transporte.success)
print("Valores óptimos de x:", resultado_transporte.x)
print("Función objetivo mínima:", resultado_transporte.fun)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0
            Iterations: 6
            Function evaluations: 4
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0
            Iterations: 3
            Function evaluations: 4
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0
            Iterations: 2
            Function evaluations: 5
            Gradient evaluations: 2
Coeficientes aleatorios para e-commerce: {'c1': 0.5706297958759382, 'c2': 0.7305088915300741}
Resultados de la optimización para e-commerce:
Estado: True
Valores óptimos de x: [0.]
Función objetivo mínima: 0.0

Coeficientes aleatorios para delivery: {'c1': 0.825888927860617, 'c2': 0.5892007204933056}
Resultados de la optimización para delivery:
Estado: True
Valores óptimos de x: [0.]
Función objetivo mínima: 0.0

Coefici

In [62]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize



# Asegurarnos de que los promedios no sean NaN ni cero
for key in promedio_tiempo:
    if np.isnan(promedio_tiempo[key]) or promedio_tiempo[key] == 0:
        raise ValueError(f"El promedio de tiempo para {key} es inválido: {promedio_tiempo[key]}")

# Definir la función de minimización para cada categoría
def minimizar_tiempo(categoria):
    # Asignar coeficientes aleatorios
    coef = {'c1': np.random.uniform(0.1, 1.0), 'c2': np.random.uniform(0.1, 1.0)}
    
    def objetivo(x):
        # x[0]: cantidad de servicios
        tiempo_total = x[0] * promedio_tiempo[categoria]
        valor_objetivo = coef['c1'] * x[0] + coef['c2'] * tiempo_total
        print(f"Evaluación de función objetivo para {categoria}: x={x}, tiempo_total={tiempo_total}, valor_objetivo={valor_objetivo}")
        return valor_objetivo
    
    # Definir las restricciones (asegurar no-negatividad)
    cons = [{'type': 'ineq', 'fun': lambda x: x[0]}]
    
    # Límites de las variables (para asegurar que sean no negativas)
    bounds = [(0, None)]
    
    # Realizar la optimización
    resultado = minimize(objetivo, x0=[1], constraints=cons, bounds=bounds, method='SLSQP', options={'disp': True})
    
    return resultado, coef

# Aplicar la minimización a cada categoría
resultado_ecommerce, coef_ecommerce = minimizar_tiempo('e-commerce')
resultado_delivery, coef_delivery = minimizar_tiempo('delivery')
resultado_transporte, coef_transporte = minimizar_tiempo('transporte')

# Imprimir los coeficientes generados y los resultados
print("Coeficientes aleatorios para e-commerce:", coef_ecommerce)
print("Resultados de la optimización para e-commerce:")
print("Estado:", resultado_ecommerce.success)
print("Valores óptimos de x:", resultado_ecommerce.x)
print("Función objetivo mínima:", resultado_ecommerce.fun)

print("\nCoeficientes aleatorios para delivery:", coef_delivery)
print("Resultados de la optimización para delivery:")
print("Estado:", resultado_delivery.success)
print("Valores óptimos de x:", resultado_delivery.x)
print("Función objetivo mínima:", resultado_delivery.fun)

print("\nCoeficientes aleatorios para transporte:", coef_transporte)
print("Resultados de la optimización para transporte:")
print("Estado:", resultado_transporte.success)
print("Valores óptimos de x:", resultado_transporte.x)
print("Función objetivo mínima:", resultado_transporte.fun)


Evaluación de función objetivo para e-commerce: x=[1.], tiempo_total=731.8548663664006, valor_objetivo=220.7729951251104
Evaluación de función objetivo para e-commerce: x=[1.00000001], tiempo_total=731.854877271888, valor_objetivo=220.77299841488437
Evaluación de función objetivo para e-commerce: x=[0.], tiempo_total=0.0, valor_objetivo=0.0
Evaluación de función objetivo para e-commerce: x=[1.49011612e-08], tiempo_total=1.0905487334227572e-05, valor_objetivo=3.289773987607813e-06
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0
            Iterations: 3
            Function evaluations: 4
            Gradient evaluations: 2
Evaluación de función objetivo para delivery: x=[1.], tiempo_total=2862.304815597125, valor_objetivo=1240.3801758677919
Evaluación de función objetivo para delivery: x=[1.00000001], tiempo_total=2862.3048582487904, valor_objetivo=1240.3801943508968
Evaluación de función objetivo para delivery: x=[7.72417707e-07], tiempo_t